This notebook contains an example of using `redbiom` through it's Python API to extract a subset of American Gut Project samples. These data are then loaded into QIIME 2 for a mini beta-diversity analysis using UniFrac. This assumes we're using a QIIME 2 2018.11 environment that additionally has `redbiom` 0.3.0 installed. The exact commands I ran to install it are:

```
$ conda install nltk
$ pip install https://github.com/biocore/redbiom/archive/0.3.0.zip
```

In [3]:
import redbiom.summarize
import redbiom.search
import redbiom.fetch
import qiime2
import pandas as pd
import skbio
from biom import load_table
import argparse
import sys
from qiime2.plugins import feature_table, diversity, emperor
pd.options.display.max_colwidth = 1000

**DIRECTORY WHERE TO EXPORT ALL CLEANED UP DATA**

In [4]:
output_dir = 'deblur_biom_wflow_output_data/'

The first thing we're going to do is gather the `redbiom` contexts. A context is roughly a set of consistent technical parameters. For example, the specific sequenced gene, the variable region within the gene, the length of the read, and how the operational taxonomic units were assessed.

The reason `redbiom` partitions data into contexts is because these technical details can lead to massive technical bias. The intention is to facilitate comparing "apples" to "apples". 

The context summarization returns a pandas `DataFrame` so it should be pretty friendly to manipulate.

In [2]:
contexts = redbiom.summarize.contexts()

In [3]:
contexts.shape

(126, 4)

At the present time, determining the context to use is a bit manual and requires some strung munging. Additional development is needed.

Let's take a look at the larger contexts.

In [4]:
contexts.sort_values('SamplesWithData', ascending=False).head()

,ContextName,SamplesWithData,FeaturesWithData,Description
106,Pick_closed-reference_OTUs-Greengenes-Illumina-16S-V4-5c6506,156761,83309,Qiita context
19,Pick_closed-reference_OTUs-Greengenes-Illumina-16S-V4-100nt-a243a1,145669,74959,Qiita context
82,Deblur-Illumina-16S-V4-100nt-fbc5b2,141809,5698310,Qiita context
2,Pick_closed-reference_OTUs-Greengenes-Illumina-16S-V4-90nt-44feac,141380,72999,Qiita context
124,Deblur-Illumina-16S-V4-90nt-99d1d8,137678,4576516,Qiita context


## Using Deblur Context

In [8]:
ctx = contexts[contexts['ContextName'] == 'Deblur-Illumina-16S-V4-100nt-fbc5b2']['ContextName'].iloc[0]

In [9]:
ctx

'Deblur-Illumina-16S-V4-100nt-fbc5b2'

**for Daniel Notes below talks about Greengenes but we are using deblur as he recommends instaed**

Breaking this name into its constiuent pieces, this is a closed reference context meaning that operational taxonomic units were assessed against a reference database and sequences which did not recruit to the reference were discarded. The reference used is Greengenes, a common 16S reference database. The gene represented by the data is the 16S SSU rRNA gene, and specifically the V4 region of the gene. Finally, the fragments represented are truncated to 100 nucleotides. (Don't worry if this is all a lot of jargon. It is a lot of jargon. Please ask questions :)

So cool, we have a "context". What can we do now? Let's search for some sample identifiers based off of the metadata (i.e., variables) associated with the samples. Specifically, let's get some skin, oral and fecal samples. Be forewarned, the metadata search uses Python's `ast` module behind the scenes, so malformed queries at present produce tracebacks.

In [10]:
study_id = 10317  # the Qiita study ID of the American Gut Project is 10317

query = "where qiita_study_id==%d" % (study_id)
results = redbiom.search.metadata_full(query)

In [11]:
len(results)

24897

In [12]:
study_id = 10317  # the Qiita study ID of the American Gut Project is 10317
results = {}
for site in ['sebum', 'saliva', 'feces']:
    query = "where qiita_study_id==%d and env_material=='%s'" % (study_id, site)
    results[site] = redbiom.search.metadata_full(query)

In [13]:
for k, v in results.items():
    print(k, len(v))

sebum 1289
saliva 1406
feces 18659


## Want to get metadata for all samples and then export to csv

In [15]:
to_keep_all = []
for k, v in results.items():
    to_keep_all.extend(list(v))

In [16]:
md_all, _ = redbiom.fetch.sample_metadata(to_keep_all, context=ctx)

In [26]:
redbiom.fetch.sample_metadata?

In [17]:
md_all.shape

(19231, 17)

In [27]:
redbiom.fetch.metadata?

In [18]:
md_all.head()

,#SampleID,anonymized_name,body_habitat,body_product,body_site,description,dna_extracted,env_biome,env_feature,env_material,env_package,host_common_name,host_taxid,qiita_study_id,sample_type,scientific_name,taxon_id
56974_10317.000047251,10317.000047251.56974,47251,UBERON:feces,UBERON:feces,UBERON:feces,American Gut Project Stool sample,Yes,dense settlement biome,human-associated habitat,feces,human-gut,human,9606,10317,Stool,human gut metagenome,408170
56974_10317.000047252,10317.000047252.56974,47252,UBERON:feces,UBERON:feces,UBERON:feces,American Gut Project Stool sample,Yes,dense settlement biome,human-associated habitat,feces,human-gut,human,9606,10317,Stool,human gut metagenome,408170
56974_10317.000047257,10317.000047257.56974,47257,UBERON:feces,UBERON:feces,UBERON:feces,American Gut Project Stool sample,Yes,dense settlement biome,human-associated habitat,feces,human-gut,human,9606,10317,Stool,human gut metagenome,408170
56974_10317.000047270,10317.000047270.56974,47270,UBERON:feces,UBERON:feces,UBERON:feces,American Gut Project Stool sample,Yes,dense settlement biome,human-associated habitat,feces,human-gut,human,9606,10317,Stool,human gut metagenome,408170
56974_10317.000047278,10317.000047278.56974,47278,UBERON:feces,UBERON:feces,UBERON:feces,American Gut Project Stool sample,Yes,dense settlement biome,human-associated habitat,feces,human-gut,human,9606,10317,Stool,human gut metagenome,408170


**exporting metadata to .csv**

#### NOTE - right now dont know how to get all metdata cols from rebiom so we will just download from qiita site

In [28]:
#md_all.to_csv('2.21.agp_metdata.csv')

The last output cell shows what these IDs look like. These are Qiita sample IDs.

Now that we have some samples, let's get some data! What we're going to do is ask `redbiom` to obtain the sample data, for our `to_keep` samples, from the context we previously selected. What's happening behind the scenes is that the API is pulling out sparse vectors corresponding to the number of individual sequences observed for each operational taxonomic unit per sample, and additionally unmunging the names (as `redbiom` normalizes sample and feature identifiers). The output is then aggregated into what's called a BIOM `Table`, which is really just a rich object wrapped around a `scipy.sparse` matrix. 

You may noice two outputs on the return. The one we're ignoring represents "ambiguous" samples. Some sample identifiers are associated with multiple sequenced samples. This is because some samples may "fail" sequencing, where they didn't yield sufficient sequence data, and were rerun. These "failures" are still represented in Qiita, but are differentiated by the actual sequencing run they were on. This doesn't matter for the moment though.

## Get the biom_table (OTU data) from deblur context

the following takes a while to run (!30 minutes)

In [66]:
biom_table, _ = redbiom.fetch.data_from_samples(ctx, to_keep_all)

In [67]:
biom_table

426648 x 19231 <class 'biom.table.Table'> with 3606467 nonzero entries (0% dense)

In [68]:
print(biom_table.head(5))

# Constructed from biom file
#OTU ID	10317.000067572.58428	10317.000035170.60177	10317.000097270.60853	10317.000041721.59400	10317.000093557.57381
TACGTAGGTGGCGAGCGTTATCCGGAATTATTGGGCGTAAAGGGTGCGTAGGCGGCTAGGCAAGTCAGACGTGAAAACCATGGGCTCAACCTGTGGACTG	0.0	0.0	0.0	0.0	0.0
TACGTAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGCGCGCAGGTGGTTTAATAAGTCTGACGTGAAAGCCCACGGCTCAACCGTGGAGGGT	0.0	0.0	0.0	0.0	0.0
TACGTAGGGTGCGAGCGTTGTCCGGAATTATTGGGCGTAAAGGGCTCGTAGGCGGTCTGTCACGTCAGGAGTGAAAACTCGGGGCCCAACCCCGAGCCTG	0.0	0.0	0.0	0.0	0.0
TACGTAGGTGGCAAGCGTTGTCCGGATTTAAAGGGTGTAAAGGGCGCGTAGGCGGGATGGCAAGTCAGATGTGAAATCCATGGGCTCAACCCATGAACTG	0.0	0.0	0.0	0.0	0.0
TACGTAGGGGGCGAGCATTGTCCGGAATCACTGGGCGTAAGGGTTCGCAGGCGGATAAACAAGTCAAGAGTAAAAGGCATGGGCTTAACCCATGTAAGCT	0.0	0.0	0.0	0.0	0.0


In [19]:
from qiime2.plugins import feature_table, diversity, emperor

In [77]:
table_ar = qiime2.Artifact.import_data('FeatureTable[Frequency]', biom_table)

In [78]:
table_ar

<artifact: FeatureTable[Frequency] uuid: feb32668-46fc-488e-a507-a0670b63727d>

## Export full_otu table so dont have to reload everything later

In [79]:
from biom import load_table

In [80]:
table_ar.export_data('full_otus')

In [81]:
!ls full_otus

feature-table.biom


In [15]:
btable = load_table('full_otus/feature-table.biom')

In [16]:
btable

426648 x 19231 <class 'biom.table.Table'> with 3606467 nonzero entries (0% dense)

In [17]:
#table_ar = qiime2.Artifact.import_data('FeatureTable[Frequency]', btable)

## Before rarification we want to remove blooms

In [73]:
import skbio
import biom
import argparse
import sys

__version__='1.0'


def trim_seqs(seqs, seqlength=100):
    """
    Trims the sequences to a given length

    Parameters
    ----------
    seqs: generator of skbio.Sequence objects

    Returns
    -------
    generator of skbio.Sequence objects
        trimmed sequences
    """

    for seq in seqs:

        if len(seq) < seqlength:
            raise ValueError('sequence length is shorter than %d' % seqlength)

        yield seq[:seqlength]


def remove_seqs(table, seqs):
    """
    Parameters
    ----------
    table : biom.Table
       Input biom table
    seqs : generator, skbio.Sequence
       Iterator of sequence objects to be removed from the biom table.

    Return
    ------
    biom.Table
    """
    filter_seqs = {str(s) for s in seqs}
    _filter = lambda v, i, m: i not in filter_seqs
    return table.filter(_filter, axis='observation', inplace=False)

In [74]:
table = load_table('full_otus/feature-table.biom')
seqs_file = 'bloom/newbloom.all.fna'

In [79]:
table

426648 x 19231 <class 'biom.table.Table'> with 3606467 nonzero entries (0% dense)

In [75]:
seqs = skbio.read(seqs_file, format='fasta')

In [76]:
length = min(map(len, table.ids(axis='observation')))
seqs = trim_seqs(seqs, seqlength=length)

In [77]:
outtable = remove_seqs(table, seqs)

In [78]:
outtable

426633 x 19231 <class 'biom.table.Table'> with 3540437 nonzero entries (0% dense)

(15 sOTUs matched bloom filter and removed)

In [80]:
table_ar = qiime2.Artifact.import_data('FeatureTable[Frequency]', outtable)

## Rarification

In [81]:
sampling_depth=1000

In [82]:
# rarefy to 1000 sequences per sample (yes, it's arbitrary)
rare_ar, = feature_table.actions.rarefy(table=table_ar, sampling_depth=sampling_depth)

In [83]:
biom_table_rar = rare_ar.view(biom.Table)

In [84]:
biom_table_rar

111507 x 18305 <class 'biom.table.Table'> with 1755714 nonzero entries (0% dense)

In [85]:
print(biom_table_rar.head(3))

# Constructed from biom file
#OTU ID	10317.000067572.58428	10317.000035170.60177	10317.000097270.60853	10317.000041721.59400	10317.000093557.57381
TACGTATGGTGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGCAGGCGGTTTGGCAAGTCTGATGTGAAAATCCGGGGCTCAACTGAGGAACTG	0.0	0.0	0.0	0.0	0.0
TACGGAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGCGCGCAGGTGGTTCCTTAAGTCTGATGTGAAAGCCCACGGCTTAACCGTGGAGGGT	0.0	0.0	0.0	0.0	0.0
TACGAAGGGGGCGAGCGTTGTTCGGAATTACTGGGCGTAAAGGGCGCGTAGGCGGCGCACCAAGTCAGGCGTGAAAGCCCTGGGCTTAACCCAGGAATTG	0.0	0.0	0.0	0.0	0.0


## converting to sparse dataframe format

In [86]:
c = biom_table_rar.matrix_data

In [87]:
c = c.transpose()

In [88]:
c

<18305x111507 sparse matrix of type '<class 'numpy.float64'>'
	with 1755714 stored elements in Compressed Sparse Column format>

In [89]:
cdf = pd.SparseDataFrame(c)

In [90]:
print(cdf.shape)

(18305, 111507)


In [91]:
cdf['sample_name'] = biom_table_rar.ids()

In [92]:
cdf.head(3)

,0,1,2,3,4,5,6,7,8,9,...,111498,111499,111500,111501,111502,111503,111504,111505,111506,sample_name
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10317.000067572.58428
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10317.000035170.60177
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10317.000097270.60853


**write biom sparse dataframe to pickle file**

In [93]:
cdf.to_pickle(output_dir + '2.21.rar1000.biom_data.pkl')

## create dna_seq lookup_id which matches the column in cdf

In [94]:
otu_ids =   biom_table_rar.ids(axis='observation')

In [95]:
otu_id_df = pd.DataFrame(otu_ids, columns=['dna_seq'])

In [96]:
otu_id_df['lookup_id'] = range(len(otu_id_df)) 

In [97]:
otu_id_df.head()

,dna_seq,lookup_id
0,TACGTATGGTGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGCAGGCGGTTTGGCAAGTCTGATGTGAAAATCCGGGGCTCAACTGAGGAACTG,0
1,TACGGAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGCGCGCAGGTGGTTCCTTAAGTCTGATGTGAAAGCCCACGGCTTAACCGTGGAGGGT,1
2,TACGAAGGGGGCGAGCGTTGTTCGGAATTACTGGGCGTAAAGGGCGCGTAGGCGGCGCACCAAGTCAGGCGTGAAAGCCCTGGGCTTAACCCAGGAATTG,2
3,TACGTAGGGTGCGAGCGTTATCCGGAATTATTGGGCGTAAAGAGCTCGTAGGCGGTTTGTCGCGTCTGTCGTGAAAGCCCGGGGCTTAACCCCGGATCTG,3
4,TACGTATGGAGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGTAGACGGACTGGCAAGTCTGATGTGAAAGGCGGGGGCTCAACCCCTGGACTG,4


**write id lookup to csv file**

In [98]:
otu_id_df.to_csv(output_dir + '2.21.rar1000.biom_ids.csv', index=False)